In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "4"
import cvxpy as cp
from utils import *
from scipy.linalg import sqrtm
import numpy as np




test_dataset = GraphDataset(f'../data/testing/ER_2000vertices_unweighted',ordered=True)


graph = test_dataset.get()

n = len(graph)

matrix = cp.Variable((n , n ), PSD=True)

cut = .25 * cp.sum(cp.multiply(graph, 1 - matrix))

problem = cp.Problem(cp.Maximize(cut), [cp.diag(matrix) == 1])

problem.solve(verbose=True,max_iters = 10000)



                                     CVXPY                                     
                                     v1.5.2                                    
(CVXPY) Sep 04 01:19:18 PM: Your problem has 4000000 variables, 2000 constraints, and 0 parameters.
(CVXPY) Sep 04 01:19:18 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 04 01:19:18 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 04 01:19:18 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Sep 04 01:19:18 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 04 01:19:18 PM: Compiling problem (target solver=SCS

14135.967572434723

In [2]:
eigenvalues, eigenvectors = np.linalg.eigh(matrix.value)
eigenvalues = np.maximum(eigenvalues, 0)
diagonal_root = np.diag(np.sqrt(eigenvalues))
solution = diagonal_root @ eigenvectors.T
size = len(solution)
partition = np.random.normal(size=size)
projections = solution.T @ partition

spins = np.sign(projections)

In [3]:
# vectors = matrix.value
# random = np.random.normal(size=vectors.shape[1])
# random /= np.linalg.norm(random, 2)

# spins = np.sign(np.dot(vectors, random))

# print(np.sign(np.dot(vectors, random)))

cut = (1/4) * np.sum( np.multiply( graph, 1 - np.outer(spins, spins) ) )
print(cut)

11261.0


In [22]:
problem.solver_stats

SolverStats(solver_name='SCS', solve_time=21.681970189, setup_time=0.481454469, num_iters=300, extra_stats={'x': array([ 1.        , -0.46005134, -0.11417227, ...,  1.        ,
        0.29207976,  1.00000001]), 'y': array([ 2.81559176e+00,  3.26566248e+00,  3.39374897e+00, ...,
        2.77065967e+00, -5.76880950e-07,  2.94830329e+00]), 's': array([0.        , 0.        , 0.        , ..., 0.99999994, 0.41306315,
       1.00001175]), 'info': {'status_val': 1, 'iter': 300, 'scale_updates': 1, 'scale': 0.33771147804621665, 'pobj': -2495.1987225341145, 'dobj': -2495.1987219823636, 'res_pri': 1.8136954520180453e-05, 'res_dual': 4.286679435652077e-06, 'gap': 5.517508725461084e-07, 'res_infeas': nan, 'res_unbdd_a': 0.0004007696875019722, 'res_unbdd_p': 0.0, 'comp_slack': 2.6284396881237626e-10, 'solve_time': 21681.970189, 'setup_time': 481.454469, 'lin_sys_time': 2264.7561059999994, 'cone_time': 18264.397310000015, 'accel_time': 187.38331300000007, 'rejected_accel_steps': 1, 'accepted_accel_